### Отбор признаков Датасета и изучение влияния на метрики модели машинного обучения

В качестве датасета выступает набор данных о стоимости автомобилей в США. 

В ходе данного задания будет произведен отбор предикторов тремя различными методами. 
- Метод рекурсивного исключения признаков 
- Метод выбора признаков на основе фильтров 
    - С применением коэффициента корреляции Пирсона
    - С применение дисперсионного анализа ANOVA 

Далее, будут обучены три модели логистической регрессии, и произведены расчеты метрик для этих трех моделей. В качестве метрики выступает MAE - средняя абсолютная ошибка. 

По результатам будут сделаны выводы. 

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression, f_classif

# Чтение файла
url = 'https://github.com/Nitys70/LESSONS_FILES/raw/refs/heads/main/ML-6/data/data_ford_price.xlsx'

data_ford = pd.read_excel(url)


# Предварительный отбор признаков (берем числовые признаки)
data_ford = data_ford[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
# Избавляемся от пропусков
data_ford.dropna(inplace = True)

# Делим датасет на таргетное значение и предикторы, 
# формируем тренировочную и тестовую выборки
y = data_ford['price']
x = data_ford.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Прибегаем к методу рекурсивного исключения признаков (RFE), формируем список признаков
estimator = LinearRegression()
selector_rfe = RFE(estimator, n_features_to_select=3, step=1)
selector_rfe = selector_rfe.fit(X_train, y_train)
 
rfe_list = list(selector_rfe.get_feature_names_out())


# Прибегаем к методу отбора признаков с помощью коэф-та корелляции Спирмена, 
# формируем список признаков
selector_f_reg = SelectKBest(f_regression, k=3)
selector_f_reg.fit(X_train, y_train)
 
f_reg_list = list(selector_f_reg.get_feature_names_out())

# Прибегаем к методу отбора признаков с помощью коэф-та корелляции Спирмена, 
# формируем список признаков
selector_f_classif = SelectKBest(f_classif, k=3)
selector_f_classif.fit(X_train, y_train)
 
f_classif_list = list(selector_f_classif.get_feature_names_out())

# На основе полученных списков формируем словарь (по нему далее пойдем циклом)
feats_top_dict = {'RFE Selection': rfe_list,
                  'KBest selection f_regression': f_reg_list,
                  'KBest selection f_classif': f_classif_list
                  }

# Задаем пустой словарь - сюда занесем результаты метрик MAE различных моделей
mae_results = dict()

# Циклом идем по созданным элементам словаря и обучаем модель, считаем метрики
for key, value in feats_top_dict.items():
    
    # Формируем DF предикторов (исходя из результатов отбора признаков), таргета
    x = data_ford[value]
    y = data_ford['price']
    
    # Делим на тренировочную и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.3, random_state=42)
    
    # Создаем модель, обучаем ее, делаем прогноз, считаем метрику
    model = LinearRegression()
    
    model.fit(X_train, y_train)
    
    y_predicted = model.predict(X_test)
    
    mae = round(mean_absolute_error(y_test, y_predicted), 3)
    
    # Заносим в словарь название метода отбора признака и MAE, полученного на этих признаках
    mae_results[key] = mae
    
    
# Формируем из полученного словаря с метриками MAE DF, фильтруем по убыванию
mae_results_df = pd.DataFrame.from_dict(mae_results, orient='index')
mae_results_df = mae_results_df.sort_values(0)
mae_results_df = mae_results_df.reset_index()
mae_results_df.columns = ['Selection method', 'MAE Value']

# Выводим на печать
display(mae_results_df)


,Selection method,MAE Value
0,KBest selection f_regression,4606.024
1,KBest selection f_classif,4606.024
2,RFE Selection,5213.692


Мы видим два одинаковых значения метрик. Вероятно, методами KBest были отобраны одни и те же признаки. Проверим это. 

In [2]:
f_reg_list == f_classif_list

True

Действительно, одинаковые метрики получены в результате идентичности отобранных придикторов.

**Вывод:**

Наилучшая метрика средней абсолютной ошибки **(MAE: 4606.024)** получена у модели, обученной на предикторах, которые мы получили при применении метода выбора признаков на основе статистических мер. Причем, при применении коэффциента корреляции Пирсона и дисперсного анализа получены идентичные значения. 

Так же, важно отметить, что прирост метрики наблюдается и относительно моделей, обученных на всех числовых признаках, лишенных мультиколлинеарности **(MAE: 4672.930)**.

Обратный эффект (ухудщение метрики) при использовании отбора предикторов методом рекурсивного исключения признаков **(MAE: 5213.692)**. Здесь значение метрики существенно снизилось. 